In [45]:
import pandas as pd
import numpy as np
import gc

from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
import pyarrow.dataset as ds
from ydata_profiling import ProfileReport

In [16]:
#데이터 분할(폴더) 구분
data_splits = ["train", "test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]
        
        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"C:/Users/dobi/Desktop/study/dacon-credit-segmentation/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            tmp = 0
            if split == 'train':
                dataset = ds.dataset(file_path, format="parquet")
                
                scanner = dataset.head(1000)
                tmp = scanner.to_pandas()
            else:
                tmp = pd.read_parquet(file_path)
                
            globals()[variable_name] = tmp
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

customer_train_07 is loaded from C:/Users/dobi/Desktop/study/dacon-credit-segmentation/train/1.회원정보/201807_train_회원정보.parquet
customer_train_08 is loaded from C:/Users/dobi/Desktop/study/dacon-credit-segmentation/train/1.회원정보/201808_train_회원정보.parquet
customer_train_09 is loaded from C:/Users/dobi/Desktop/study/dacon-credit-segmentation/train/1.회원정보/201809_train_회원정보.parquet
customer_train_10 is loaded from C:/Users/dobi/Desktop/study/dacon-credit-segmentation/train/1.회원정보/201810_train_회원정보.parquet
customer_train_11 is loaded from C:/Users/dobi/Desktop/study/dacon-credit-segmentation/train/1.회원정보/201811_train_회원정보.parquet
customer_train_12 is loaded from C:/Users/dobi/Desktop/study/dacon-credit-segmentation/train/1.회원정보/201812_train_회원정보.parquet
credit_train_07 is loaded from C:/Users/dobi/Desktop/study/dacon-credit-segmentation/train/2.신용정보/201807_train_신용정보.parquet
credit_train_08 is loaded from C:/Users/dobi/Desktop/study/dacon-credit-segmentation/train/2.신용정보/201808_train_신용정보.parq

1748

In [17]:
# 데이터 유형별 설정 
info_categories = ["customer", "credit", "sales", "billing", "balance", "channel", "marketing", "performance"]

# 월 설정
months = ['07', '08', '09', '10', '11', '12']

In [18]:
#### Train ####

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")


customer_train_df = train_dfs["customer_train_df"]
credit_train_df   = train_dfs["credit_train_df"]
sales_train_df    = train_dfs["sales_train_df"]
billing_train_df  = train_dfs["billing_train_df"]
balance_train_df  = train_dfs["balance_train_df"]
channel_train_df  = train_dfs["channel_train_df"]
marketing_train_df= train_dfs["marketing_train_df"]
performance_train_df = train_dfs["performance_train_df"]

gc.collect()

customer_train_df is created with shape: (6000, 78)
credit_train_df is created with shape: (6000, 42)
sales_train_df is created with shape: (6000, 406)
billing_train_df is created with shape: (6000, 46)
balance_train_df is created with shape: (6000, 82)
channel_train_df is created with shape: (6000, 105)
marketing_train_df is created with shape: (6000, 64)
performance_train_df is created with shape: (6000, 49)


0

In [19]:
#### Test ####

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")


customer_test_df = test_dfs["customer_test_df"]
credit_test_df   = test_dfs["credit_test_df"]
sales_test_df    = test_dfs["sales_test_df"]
billing_test_df  = test_dfs["billing_test_df"]
balance_test_df  = test_dfs["balance_test_df"]
channel_test_df  = test_dfs["channel_test_df"]
marketing_test_df= test_dfs["marketing_test_df"]
performance_test_df = test_dfs["performance_test_df"]

gc.collect()

customer_test_df is created with shape: (600000, 77)
credit_test_df is created with shape: (600000, 42)
sales_test_df is created with shape: (600000, 406)
billing_test_df is created with shape: (600000, 46)
balance_test_df is created with shape: (600000, 82)
channel_test_df is created with shape: (600000, 105)
marketing_test_df is created with shape: (600000, 64)
performance_test_df is created with shape: (600000, 49)


0

In [20]:
#### Train ####

train_df = customer_train_df.merge(credit_train_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: train_step1, shape:", train_df.shape)
del customer_train_df, credit_train_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장
merge_list = [
    ("sales_train_df",    "Step2"),
    ("billing_train_df",  "Step3"),
    ("balance_train_df",  "Step4"),
    ("channel_train_df",  "Step5"),
    ("marketing_train_df","Step6"),
    ("performance_train_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    train_df = train_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: train_{step}, shape:", train_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

Step1 저장 완료: train_step1, shape: (6000, 118)
Step2 저장 완료: train_Step2, shape: (6000, 522)
Step3 저장 완료: train_Step3, shape: (6000, 566)
Step4 저장 완료: train_Step4, shape: (6000, 646)
Step5 저장 완료: train_Step5, shape: (6000, 749)
Step6 저장 완료: train_Step6, shape: (6000, 811)
최종 저장 완료: train_최종, shape: (6000, 858)


In [21]:
#### Test ####

test_df = customer_test_df.merge(credit_test_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: test_step1, shape:", test_df.shape)
del customer_test_df, credit_test_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장
merge_list = [
    ("sales_test_df", "Step2"),
    ("billing_test_df", "Step3"),
    ("balance_test_df", "Step4"),
    ("channel_test_df", "Step5"),
    ("marketing_test_df", "Step6"),
    ("performance_test_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    test_df = test_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: test_{step}, shape:", test_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

Step1 저장 완료: test_step1, shape: (600000, 117)
Step2 저장 완료: test_Step2, shape: (600000, 521)
Step3 저장 완료: test_Step3, shape: (600000, 565)
Step4 저장 완료: test_Step4, shape: (600000, 645)
Step5 저장 완료: test_Step5, shape: (600000, 748)
Step6 저장 완료: test_Step6, shape: (600000, 810)
최종 저장 완료: test_최종, shape: (600000, 857)


In [22]:
feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]

X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()

# 타깃 라벨 인코딩
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

In [23]:
print(X.dtypes.value_counts())

int64      747
float64     61
object      48
Name: count, dtype: int64


In [24]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

X_test = test_df.copy()

encoders = {}  # 각 컬럼별 encoder 저장

for col in categorical_features:
    le_train = LabelEncoder()
    X[col] = le_train.fit_transform(X[col])
    encoders[col] = le_train
    unseen_labels_val = set(X_test[col]) - set(le_train.classes_)
    if unseen_labels_val:
        le_train.classes_ = np.append(le_train.classes_, list(unseen_labels_val))
    X_test[col] = le_train.transform(X_test[col])

In [25]:
gc.collect()

0

결측치 비율 탐사

In [26]:
missing_df = X.isnull().sum().to_frame(name='missing_count')
missing_df['missing_ratio (%)'] = (missing_df['missing_count'] / len(X)) * 100
missing_df = missing_df[missing_df['missing_count'] > 0]
print(missing_df)

                missing_count  missing_ratio (%)
최종유효년월_신용_이용가능            454           7.566667
최종유효년월_신용_이용             1236          20.600000
최종카드발급일자                  101           1.683333
RV신청일자                   4816          80.266667
최종카드론_금융상환방식코드           4896          81.600000
최종카드론_대출일자               4968          82.800000
연체일자_B0M                 5995          99.916667
혜택수혜율_R3M                1147          19.116667
혜택수혜율_B0M                1300          21.666667


In [27]:
print(X.dtypes.value_counts())

int64      747
float64     61
int32       48
Name: count, dtype: int64


Mutual Information 계산

In [28]:
# 결측치가 50% 이상인 컬럼만 제거
X_clean = X.dropna(axis=1)



# Mutual Information 계산
X_num = X_clean.select_dtypes(include=['number'])
mi_scores = mutual_info_classif(X_num, y_encoded, random_state=42)

# 컬럼 이름과 MI 값을 묶어서 리스트로 만들고, MI 기준으로 내림차순 정렬
mi_result = sorted(zip(X_num.columns, mi_scores), key=lambda x: x[1], reverse=True)
# mi_result → DataFrame으로 변환
mi_df = pd.DataFrame(mi_result, columns=['feature', 'mutual_info'])

# 범주 구분 함수 정의
def categorize_mi(mi):
    if mi >= 0.1:
        return 'high'
    elif 0.01 < mi < 0.1:
        return 'middle'
    else:
        return 'low'

# 범주 컬럼 추가
mi_df['category'] = mi_df['mutual_info'].apply(categorize_mi)

# 결과 출력
print(mi_df['category'].value_counts())

category
low       435
middle    368
high       44
Name: count, dtype: int64


모든 값이 동일한 변수 있는지 확인 후 constant_columns에 담기

In [32]:
constant_columns = X.columns[X.nunique(dropna=False) == 1]
print(len(constant_columns))

152


보고서 생성 (uniform 변수 제외)

In [54]:
tmp_X = X.drop(columns=constant_columns)
"""
profile = ProfileReport(
    tmp_X,
    minimal=True
)
profile.to_file('profile.html')
"""

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 704/704 [00:03<00:00, 195.86it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

각 변수별 0의 비율을 조사하고 시리즈에 넣어둠

In [63]:
zero_ratio = (tmp_X == 0).sum() / len(X)
high_zero_columns = zero_ratio[zero_ratio >= 0.5]
extreme_high_zero_columns = zero_ratio[zero_ratio >= 0.9]
print(high_zero_columns.sort_values(ascending=False))
print(len(high_zero_columns))
print(len(extreme_high_zero_columns))

할부건수_유이자_14M_R12M    0.999833
컨택건수_이용유도_당사앱_R6M    0.999833
IB문의건수_CA_R6M        0.999833
할부건수_14M_R12M        0.999833
유효카드수_체크_가족          0.999833
                       ...   
RP유형건수_B0M           0.538000
연체입금원금_B0M           0.526833
RP건수_B0M             0.524667
RP금액_B0M             0.521500
이용후경과월_신판            0.516500
Length: 470, dtype: float64
470
278


상관계수가 0.9이상인 쌍들을 찾아내어 저장

In [67]:
corr_matrix = tmp_X.corr(method="pearson")


# 자기 자신과의 상관은 제외 (1.0 값)
# → 삼각 행렬 중 하나만 보자 (예: 상삼각)
high_corr_pairs = []
equal_corr_pairs = []

# 상삼각 행렬만 탐색 (중복 방지)
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        col1 = corr_matrix.columns[i]
        col2 = corr_matrix.columns[j]
        corr_value = corr_matrix.iloc[i, j]
        if 0.9 <= abs(corr_value):
            high_corr_pairs.append((col1, col2, corr_value))
        if abs(corr_value) == 1.0:
            equal_corr_pairs.append((col1, col2, corr_value))

# 결과를 DataFrame으로 변환
high_corr_df = pd.DataFrame(high_corr_pairs, columns=["Variable_1", "Variable_2", "Correlation"])
equal_corr_df = pd.DataFrame(equal_corr_pairs, columns=["Variable_1", "Variable_2", "Correlation"])

# 확인
print(high_corr_df)


             Variable_1          Variable_2  Correlation
0                  기준년월            연체일자_B0M     0.998933
1           소지카드수_유효_신용            연체일자_B0M     0.954671
2               입회일자_신용          입회경과개월수_신용    -0.999119
3               회원여부_연체         이용횟수_연체_B0M     1.000000
4            이용거절여부_카드론             카드론동의여부    -1.000000
..                  ...                 ...          ...
781  잔액_신판ca평균한도소진율_r6m  잔액_신판ca최대한도소진율_r3m     0.980885
782  잔액_신판ca최대한도소진율_r6m  잔액_신판ca평균한도소진율_r3m     0.970337
783  잔액_신판ca최대한도소진율_r6m  잔액_신판ca최대한도소진율_r3m     0.979701
784  잔액_신판ca평균한도소진율_r3m  잔액_신판ca최대한도소진율_r3m     0.993383
785         변동률_RV일시불평잔          변동률_RVCA평잔     0.934561

[786 rows x 3 columns]


In [68]:
print(equal_corr_df)

         Variable_1         Variable_2  Correlation
0           회원여부_연체        이용횟수_연체_B0M          1.0
1       유효카드수_체크_가족      이용가능카드수_체크_가족          1.0
2         기본연회비_B0M     청구금액_기본연회비_B0M          1.0
3         제휴연회비_B0M     청구금액_제휴연회비_B0M          1.0
4     자발한도감액횟수_R12M      자발한도감액금액_R12M          1.0
5        최종이용일자_카드론         최종카드론_대출일자          1.0
6   이용금액_부분무이자_R12M   할부금액_부분_12M_R12M          1.0
7    이용건수_부분무이자_R3M    이용개월수_부분무이자_R3M          1.0
8     할부건수_14M_R12M  할부건수_유이자_14M_R12M          1.0
9        승인거절건수_B0M    승인거절건수_한도초과_B0M          1.0
10     대표청구서수령지구분코드            청구서수령방법          1.0
11  증감율_이용건수_카드론_전월    증감율_이용금액_카드론_전월          1.0
